In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_07 import *

# Layer wise sequential unit variance

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [5]:
nfs = [8,16,32,64,64]

In [6]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks // 2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    def forward(self, x):
        return self.relu(self.conv(x))
    
    @property
    def bias(self): 
        return -self.relu.sub
    @bias.setter
    def bias(self,v):
        self.relu.sub = -v
    @property
    def weight(self):
        return self.conv.weight

In [7]:
learn, run = get_learn_run(nfs, data, .6, ConvLayer, cbs=cbfs)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [8]:
run.fit(2, learn)

train: [1.7722040625, tensor(0.3779, device='cuda:0')]
valid: [0.346475927734375, tensor(0.8934, device='cuda:0')]
train: [0.2584407421875, tensor(0.9205, device='cuda:0')]
valid: [0.1631513916015625, tensor(0.9492, device='cuda:0')]


Getting out a single mini batch

In [9]:
#export
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs:
        cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

In [10]:
xb,yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [11]:
#export
def find_modules(m, cond):
    if cond(m): 
        return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [12]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))

In [13]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

In [14]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean, hook.std = d.mean().item(), d.std().item()

In [15]:
mdl = learn.model.cuda()

In [16]:
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.6988096237182617 1.0726783275604248
0.5895874500274658 1.5364489555358887
0.4192093312740326 1.2666932344436646
0.3908877968788147 1.0187383890151978
0.5606163740158081 1.3941950798034668


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [17]:
#export
def lsuv_module(m, xb):
    h = Hook(m, append_stat)
    while mdl(xb) is not None and abs(h.mean) > 1e-3:
        m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std - 1) > 1e-3:
        m.weight.data /= h.std
    
    h.remove()
    return h.mean, h.std

We execute that initialization on all the conv layers in order:

In [18]:
for m in mods:
    print(lsuv_module(m, xb))

(-0.04904128611087799, 1.000531792640686)
(-0.1771754026412964, 1.0000052452087402)
(-0.07002484053373337, 1.0001193284988403)
(-0.04661491885781288, 1.000038981437683)
(-0.23717892169952393, 1.000050663948059)


In [19]:
%time run.fit(2, learn)

train: [nan, tensor(0.1499, device='cuda:0')]
valid: [nan, tensor(0.0991, device='cuda:0')]
train: [nan, tensor(0.0986, device='cuda:0')]
valid: [nan, tensor(0.0991, device='cuda:0')]
CPU times: user 3.7 s, sys: 361 ms, total: 4.06 s
Wall time: 3.71 s


**LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.**

# Export

In [20]:
!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to exp/nb_07a.py
